In [205]:
import os
import sklearn
import numpy as np
from numpy import unwrap, diff, abs, angle
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from sklearn.utils import shuffle
import scipy
from scipy.signal import butter, filtfilt, hilbert
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.layers import Dense,  BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import mne
from mne.preprocessing import ICA
import pywt
from scipy.stats import skew, kurtosis
from scipy.signal import spectrogram

Preproccesing Functions

In [206]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def denoise_data(df, col_names, n_clusters):
    df_denoised = df.copy()
    for col_name, k in zip(col_names, n_clusters):
        df_denoised[col_name] = pd.to_numeric(df_denoised[col_name], errors='coerce') # Convert column to numeric format
        X = df_denoised.select_dtypes(include=['float64', 'int64']) # Select only numeric columns
        clf = KNeighborsRegressor(n_neighbors=k, weights='uniform') # Fit KNeighborsRegressor
        clf.fit(X.index.values[:, np.newaxis], X[col_name])
        y_pred = clf.predict(X.index.values[:, np.newaxis]) # Predict values 
        df_denoised[col_name] = y_pred
    return df_denoised

def z_score(df, col_names):
    df_standard = df.copy()
    for col in col_names:
        df_standard[col] = (df[col] - df[col].mean()) / df[col].std()
    return df_standard

def custom_detrend(df, col_names):
    df_detrended = df.copy()
    for col in col_names:
        y = df_detrended[col]
        x = np.arange(len(y))
        p = np.polyfit(x, y, 1)
        trend = np.polyval(p, x)
        detrended = y - trend
        df_detrended[col] = detrended
    return df_detrended

def preprocess(df, col_names, n_clusters):
    df_new = df.copy()
    df_new = denoise_data(df, col_names, n_clusters)
    df_new = z_score(df_new, col_names)
    df_new = custom_detrend(df_new, col_names)
    return df_new

def df_to_raw(df, sfreq=250):
    info = mne.create_info(ch_names=list(df.columns), sfreq=sfreq, ch_types=['eeg'] * df.shape[1])
    raw = mne.io.RawArray(df.T.values * 1e-6, info)  # Converting values to Volts from microvolts for MNE
    return raw

def reject_artifacts(df, channel):
    threshold_factor = 3
    median = df[channel].median()
    mad = np.median(np.abs(df[channel] - median))
    spikes = np.abs(df[channel] - median) > threshold_factor * mad
    x = np.arange(len(df[channel]))
    cs = CubicSpline(x[~spikes], df[channel][~spikes]) # Interpolate using Cubic Spline
    interpolated_values = cs(x)
    interpolated_values[spikes] *= 0.1  # Make interpolated values 0.1 times smaller
    # Check each interpolated value's difference from median and compare to the threshold
    spike_values = np.abs(interpolated_values - median) > threshold_factor * mad
    interpolated_values[spike_values] *= 0.01 
    df[channel] = interpolated_values
    return df


Define the dataset

In [207]:
folder_name = 'i'
selected_columns = ['Fz', 'FC1', 'FC2', 'C3', 'Cz', 'C4', 'CPz', 'Pz']
duration = 40 
raw=[]
event=[]
BP=[]
PP=[]
if os.path.exists(folder_name) and os.path.isdir(folder_name):
    for file_name in os.listdir(folder_name):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_name, file_name)
            s_temp = pd.read_csv(file_path, header=None)
            inst = s_temp.iloc[:, 17]
            df_temp = s_temp.iloc[:, :8]
            # print(df_temp.shape)
            # df_temp.plot(figsize=(10, 8))
            # plt.show()
            raw.append(df_temp)
            event.append(inst)
            
            # 1. Band Pass
            raw_bp = np.copy(df_temp)
            for column in range(8):
                raw_bp[:, column] = butter_bandpass_filter(raw_bp[:, column], lowcut=.4, highcut=40, fs=250) 
            print(raw_bp.shape)
            # plt.plot(raw_bp)
            # plt.show()
            
            # 2. Artifact rejection
            BP_artifact_RJ = np.copy(raw_bp)
            for channel in range (8):
                BP_artifact_RJ= reject_artifacts(pd.DataFrame(BP_artifact_RJ), channel)
            # plt.plot(BP_artifact_RJ)
            # plt.show()
            
            # 3. Smoothing
            BP_artifact_RJ_SM=BP_artifact_RJ.copy()
            window_size = 10 
            for channel in range (8):
                # channel_data = BP_artifact_RJ_SM[channel, :]
                BP_artifact_RJ_SM= BP_artifact_RJ_SM.rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(method='ffill')
            # plt.plot(BP_artifact_RJ_SM)
            # plt.show()
            BP.append(BP_artifact_RJ_SM)
            
            # 4. Denoising and other preprocessing
            BP_artifact_RJ_SM.columns = selected_columns
            eeg_df_denoised = preprocess(pd.DataFrame(BP_artifact_RJ_SM), col_names=selected_columns, n_clusters=[50]*len(selected_columns))
            # eeg_df_denoised.plot(subplots=True, figsize=(15, 10), title='Denoised EEG Data')
            # plt.show()
            PP.append(eeg_df_denoised)

(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)


In [209]:
fs=250
B_N=int(len(PP)) #Number of blocks
PP_NP=np.array(PP) #shape: (B_N, 10000, 8=Channel Numbers)
event=np.array(event).reshape(B_N*(df_temp.shape[0]), 1) # df_temp.shape[0]=10000
denoised=PP_NP.reshape(B_N*(df_temp.shape[0]), 8) # seprate each blocks' signal 
pp_sig_event=np.concatenate((denoised, event), axis=1) 
labels=[] 
face = [] #lable=0
scene=[]#lable=1
# Aassuming correctness for the human behavior
for i in range(len(pp_sig_event)): #len(pp_sig_event) = the whole sample points, (df_temp.shape[0]*B_N)
    if 'M' in pp_sig_event[i, 8] or 'F' in pp_sig_event[i, 8]:
        face.append(pp_sig_event[i])
        labels.append(0)
    else:
        scene.append(pp_sig_event[i]) 
        labels.append(1)        
face = np.array(face)
scene = np.array(scene)
labels=np.array(labels) 
                 
print('event', event.shape,  'denoised',  denoised.shape, 'pp_sig_event', pp_sig_event.shape, 'face', face.shape, 'scene', scene.shape, 'labels', labels.shape)  
#denoised is all the denoised data with shape: (df_temp.shape[0]*B_N, 8)     
# event is all the events with shape: (df_temp.shape[0]*B_N, 1)                                                                                                            

event (120000, 1) denoised (120000, 8) pp_sig_event (120000, 9) face (50000, 9) scene (70000, 9) labels (120000,)


In [210]:
label=labels.reshape(int(labels.shape[0]/fs), fs)
Y=np.squeeze(label[:,0])

frequency_bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 14),
    'beta': (14, 30),
    'gamma': (30, 40),
     }

def apply_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal
denoised_reshaped = denoised.reshape(int(denoised.shape[0]/250), 250, 8)

# def extract_statistical_features_from_amplitude(coefficients):
#     """Extract statistical features from the amplitude of wavelet coefficients."""
#     amplitude = np.abs(coefficients.flatten())  # Get the amplitude and flatten
#     # Extract features from amplitude
#     mean_amp = np.mean(amplitude)
#     variance_amp = np.var(amplitude)
#     skewness_amp = skew(amplitude)
#     kurtosis_amp = kurtosis(amplitude)
#     return [mean_amp, variance_amp, skewness_amp, kurtosis_amp]

# features_combined = []
# for segment in denoised_reshaped:
#     segment_features = []
#     for channel_data in segment.T:  # Going through each channel
#         # For each frequency band, apply the bandpass filter and then extract wavelet features
#         for band, (low, high) in frequency_bands.items():
#             filtered_signal = apply_bandpass_filter(channel_data, low, high, fs)
#             coefficients, frequencies = pywt.cwt(filtered_signal, scales=np.arange(1, 50), wavelet='cmor')
#             # Extract statistical features from the amplitude
#             amp_features = extract_statistical_features_from_amplitude(coefficients)
#             segment_features.extend(amp_features)            
#     features_combined.append(segment_features)
# features_combined_np = np.array(features_combined)
# print(features_combined_np.shape)
# wavelet_np=features_combined_np.reshape(features_combined_np.shape[0], 8, int(features_combined_np.shape[1]/8))

In [211]:
# Wavelet combined with frequency bands
features_combined = []

for segment in denoised_reshaped:
    segment_features = []
    for channel_data in segment.T:  # Going through each channel
        
        # For each frequency band, apply the bandpass filter and then extract wavelet features
        for band, (low, high) in frequency_bands.items():
            filtered_signal = apply_bandpass_filter(channel_data, low, high, fs)
            coeffs = pywt.wavedec(filtered_signal, 'db4', level=4)
            coeff_array = np.concatenate(coeffs)
            amplitude = np.abs(coeff_array)
            for coeff in coeffs:
                segment_features.extend([np.mean(amplitude), np.var(amplitude), skew(amplitude), kurtosis(amplitude)])
                
    features_combined.append(segment_features)    
features_combined_np = np.array(features_combined)



In [213]:
# wavelet_np=features_combined_np.reshape(features_combined_np.shape[0], 8, int(features_combined_np.shape[1]/8))

# combined_features = np.concatenate([wavelet_np], axis=2)
print(features_combined_np.shape)  # Should print (1600, 8, 11)
# af=features_combined_np.reshape(int(features_combined_np.shape[0]), int(8*features_combined_np.shape[2]))
af=features_combined_np
af, Y = shuffle(af, Y)
print(af.shape, Y.shape)
# Balance the dataset
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(af, Y)
X_resampled= X_resampled.astype(np.float32)
y_resampled = y_resampled.astype(np.int32)

# Split X and y into training and testing sets
# X_touched, X_untouch, y_touch, y_untouch = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.1, random_state=42)

# Convert y_train and y_test to categorical format for Keras
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)
# y_untouch=tf.keras.utils.to_categorical(y_untouch, num_classes=2)


# Convert the data to a numerical type (float)
X_train = X_train.astype(np.float64)
print(X_train.shape)

# Convert one-hot-encoded labels to integer-encoded labels
y_train = np.argmax(y_train, axis=-1)
y_test = np.argmax(y_test, axis=-1)
# y_untouch = np.argmax(y_untouch, axis=-1)
print(y_train.shape, y_test.shape)


print('X_train:', X_train.shape, 'y_train:', y_train.shape, 'X_test:', X_test.shape, 'y_test:',
      y_test.shape, 'X_untouch:', 'y_untouch:' )

(480, 800)
(480, 800) (480,)
(504, 800)
(504,) (56,)
X_train: (504, 800) y_train: (504,) X_test: (56, 800) y_test: (56,) X_untouch: y_untouch:


In [214]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict class labels for the test set
y_pred = clf.predict(X_test)

# Generate and print classification report
report = classification_report(y_test, y_pred)
print(report)

# Print accuracy score
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.50      0.74      0.60        23
           1       0.73      0.48      0.58        33

    accuracy                           0.59        56
   macro avg       0.61      0.61      0.59        56
weighted avg       0.63      0.59      0.59        56

Accuracy: 0.5892857142857143


In [215]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Define the objective function
def objective(trial):
    # Define search space
    n_estimators = trial.suggest_int('n_estimators', 100, 150)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    class_weight = trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample'])

    clf = RandomForestClassifier(
        n_estimators=n_estimators, 
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42,
        criterion=criterion,
        class_weight= class_weight
    )

    # Use StratifiedKFold cross-validation
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    return cross_val_score(clf, X_train, y_train, cv=stratified_kfold, n_jobs=-1).mean()

# Create a study object and specify the direction is maximize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters
print(study.best_trial.params)




[I 2023-10-09 19:19:29,750] A new study created in memory with name: no-name-3beb3da4-ae0f-4191-a27b-e0a0b293ea23
[I 2023-10-09 19:19:30,921] Trial 0 finished with value: 0.5993267326732674 and parameters: {'n_estimators': 127, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'auto', 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5993267326732674.
[I 2023-10-09 19:19:31,377] Trial 1 finished with value: 0.5952079207920793 and parameters: {'n_estimators': 110, 'min_samples_split': 4, 'min_samples_leaf': 7, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5993267326732674.
[I 2023-10-09 19:19:31,880] Trial 2 finished with value: 0.5952871287128714 and parameters: {'n_estimators': 142, 'min_samples_split': 9, 'min_samples_leaf': 9, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5993267326732674.
[I 2023-10-09 19:19:32,394] 

{'n_estimators': 120, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced'}


In [216]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Splitting the data
# X_train_split, X_val, y_train_split, y_val = train_test_split(
#     X_train, y_train, test_size=0.2, random_state=42)

# Use best hyperparameters from the Optuna optimization (replace these with actual best params)
best_params = study.best_trial.params

clf_best = RandomForestClassifier(**best_params, random_state=42)
clf_best.fit(X_train, y_train)

# Predict on the validation set
y_pred = clf_best.predict(X_test)

# Generate and print classification report
report = classification_report(y_test, y_pred)
print(report)

# Print accuracy score
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.50      0.78      0.61        23
           1       0.75      0.45      0.57        33

    accuracy                           0.59        56
   macro avg       0.62      0.62      0.59        56
weighted avg       0.65      0.59      0.58        56

Accuracy: 0.5892857142857143


In [217]:
from sklearn import svm
from joblib import dump
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
# Create a linear SVM classifier
clf = svm.SVC(kernel='linear')

# Train the classifier
clf.fit(X_train, y_train)
# Save the model to disk
filename = 'C:/Users/tnlab/OneDrive/Documents/GitHub/Neurofeedback-Based-BCI/SVM for Unicorn Data/my_svm_model.joblib'
dump(clf, filename)

# Make predictions on the test set
y_pred = clf.predict(X_test)

print('Model accuracy: ', accuracy_score(y_test, y_pred))
report_svm_matrix = classification_report(y_test, y_pred)
print("Classification Report:")
print(report_svm_matrix)
report_svm = classification_report(y_test, y_pred, output_dict=True)

report_df_svm = pd.DataFrame(report_svm).transpose()
report_df_svm.to_excel(f"svm_classification_report_{folder_name}.xlsx", index=True)

Model accuracy:  0.5357142857142857
Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.57      0.50        23
           1       0.63      0.52      0.57        33

    accuracy                           0.54        56
   macro avg       0.54      0.54      0.53        56
weighted avg       0.56      0.54      0.54        56



In [86]:
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Define the objective function
def objective(trial):
    # Define search space
    C = trial.suggest_loguniform('C', 1e-10, 1e10)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 1, 5)
    else:
        degree = 1  # default value, won't be used
    coef0 = trial.suggest_float('coef0', -1.0, 1.0)
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

    # Create SVM classifier
    clf = SVC(C=C, kernel=kernel, degree=degree, coef0=coef0, gamma=gamma, random_state=42)

    # Use StratifiedKFold cross-validation
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    return cross_val_score(clf, X_train, y_train, cv=stratified_kfold, n_jobs=-1).mean()

# Create a study object and specify the direction is maximize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters
print(study.best_trial.params)


[I 2023-10-09 18:02:31,953] A new study created in memory with name: no-name-883a66fd-3ec7-44b3-b337-2e21922ac9f2
C:\Users\tnlab\AppData\Local\Temp\ipykernel_16848\3781216325.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-10, 1e10)
[I 2023-10-09 18:02:32,028] Trial 0 finished with value: 0.6566336633663367 and parameters: {'C': 37736667.2223723, 'kernel': 'rbf', 'coef0': 0.8298523640508921, 'gamma': 'scale'}. Best is trial 0 with value: 0.6566336633663367.
C:\Users\tnlab\AppData\Local\Temp\ipykernel_16848\3781216325.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-10, 1e10)
